In [ ]:
import geemap
import ee
geemap.set_proxy(port=4780)
ee.Initialize()
from keras.models import load_model
model = load_model(r'[filename]')
####################################
# filename of the best model
####################################
config = model.get_config()
layers = config['layers']
import numpy as np
def cloud_free(image):
    """
        inputs: image
        outputs: cloud-free image
    """
    cloud = image.select('QA_PIXEL').rightShift(1).bitwiseAnd(15)
    image = image.select('SR_B.').multiply(0.0000275).add(-0.2).addBands(image.select('QA_PIXEL').rightShift(7).bitwiseAnd(1))
    image_freecloud = image.updateMask(cloud.expression('b==0',{'b':cloud}))
    return image_freecloud
def summer_img(l8, st_year, ed_year, path, row):
    """
        inputs: imagecollection, start year, end year, path, and row
        outputs: imagecollection
    """
    landsat = l8.filter(ee.Filter.eq('WRS_PATH', int(path)))\
            .filter(ee.Filter.eq('WRS_ROW', int(row)))\
            .filter(ee.Filter.calendarRange(int(st_year),int(ed_year),'year'))\
            .filter(ee.Filter.calendarRange(6,8,'month'))\
            .sort('CLOUD_COVER')
    return landsat
def summer_img_bounds(l8,st_year,ed_year,bounds):
    """
        inputs: imagecollection, start year, end year, bound
        outputs: imagecollection
    """
    landsat = l8.filter(ee.Filter.calendarRange(int(st_year),int(ed_year),'year'))\
            .filter(ee.Filter.calendarRange(6,8,'month'))\
            .filterBounds(bounds).sort('CLOUD_COVER')
    return landsat
def features(image):
    """
        inputs: image in gee format
        outputs: image with feature bands in gee format
    """
    weights1 = [[0,0,-1,0,0],
                   [0,0, 0,0,0],
                   [0,0, 2,0,0],
                   [0,0, 0,0,0],
                   [0,0, -1,0,0]]
    weights2 = [[-1,0,0,0,0],
                [0,0, 0,0,0],
                [0,0, 2,0,0],
                [0,0, 0,0,0],
                [0,0,0,0, -1]]
    weights3 = [[0,0,0,0,-1],
               [0,0, 0,0,0],
               [0,0, 2,0,0],
               [0,0, 0,0,0],
               [-1,0, 0,0,0]]
    weights4 = [[0,0,0,0,0],
               [0,0, 0,0,0],
               [-1,0, 2,0,-1],
               [0,0, 0,0,0],
               [0,0, 0,0,0]]
    Blue = image.select('SR_B2').rename('B')
    Green = image.select('SR_B3').rename('G')
    Red = image.select('SR_B4').rename('R')
    NIR = image.select('SR_B5').rename('N')
    SWIR = image.select('SR_B6').rename('SW')
    ndwi = image.normalizedDifference(['SR_B3','SR_B6']).rename('mndwi')
    smooth1 = ndwi.convolve(ee.Kernel.fixed(weights=ee.List(weights1)))
    smooth2 = ndwi.convolve(ee.Kernel.fixed(weights=ee.List(weights2)))
    smooth3 = ndwi.convolve(ee.Kernel.fixed(weights=ee.List(weights3)))
    smooth4 = ndwi.convolve(ee.Kernel.fixed(weights=ee.List(weights4)))
    smooth = ee.Image([smooth1,smooth2,smooth3,smooth4]).reduce(ee.Reducer.max())
    smooth = smooth.rename('line')
    dem = image.select('elevation').rename('dem')
    image = Blue.addBands(Green).addBands(Red).addBands(NIR).addBands(SWIR).addBands(ndwi).addBands(smooth).addBands(dem)
    return image
def pixel_based_CNN(image):
    """
        inputs: image
        outputs: predicted image
    """
    in_names = [i['inbound_nodes'] for i in layers]
    out_names = [i['name'] for i in layers]
    new_bandnames = ['B','G','R','N','SW','mndwi','line','dem']
    results_layers = [image]
    for layer in layers[1:]:
        if layer['class_name'] == 'Conv2D':
            idx = out_names.index(layer['name'])
            idx = out_names.index(in_names[idx][0][0][0])
            tmp1 = results_layers[idx]
            w,b = model.get_layer(layer['config']['name']).get_weights()
            bands = [bd['id'] for bd in tmp1.getInfo()['bands']]
            kx,ky = layer['config']['kernel_size']
            print(bands)
            bef_lon, aft_lon = w.shape[-2:]
            kernel = ee.Kernel.fixed(kx,ky,ee.List(w[:,:,0,0].tolist()),-1,-1,False)
            tmp=tmp1.select('B').convolve(kernel).add(b[0])
            for j,band_name,bias in zip(range(1,bef_lon),bands[1:],b[1:]):
                kernel = ee.Kernel.fixed(kx,ky,ee.List(w[:,:,j,0].tolist()),-1,-1,False)
                tmp = tmp.add(tmp1.select(band_name).convolve(kernel).add(bias))
            conv1 = tmp
            for i in range(1,aft_lon):
                kernel = ee.Kernel.fixed(kx,ky,ee.List(w[:,:,0,i].tolist()),-1,-1,False)
                tmp=tmp1.select(bands[0]).convolve(kernel)
                for j,band_name,bias in zip(range(1,bef_lon),bands[1:],b[1:]):
                    kernel = ee.Kernel.fixed(kx,ky,ee.List(w[:,:,j,i].tolist()),-1,-1,False)
                    tmp = tmp.add(tmp1.select(band_name).convolve(kernel).add(bias))
                if layer['config']['activation']== 'relu':
                    tmp = relu(tmp)
                else:
                    pass
                conv1=ee.Image.cat([conv1,tmp])
            results_layers.append(conv1)
            print(layer['class_name'],len(results_layers))
        elif layer['class_name'] == 'TensorFlowOpLayer':
            idx = out_names.index(layer['name'])
            idx = out_names.index(in_names[idx][0][0][0])
            tmp1 = results_layers[idx]
            results_layers.append(tmp1)
            print(layer['class_name'],len(results_layers))
        elif layer['class_name'] == 'Concatenate':
            idx = out_names.index(layer['name'])
            idx1 = out_names.index(in_names[idx][0][0][0])
            idx2 = out_names.index(in_names[idx][0][1][0])
            tmp1 = results_layers[idx1]
            tmp2 = results_layers[idx2]
            results_layers.append(ee.Image.cat([tmp1,tmp2.rename(new_bandnames)]))
            print(layer['class_name'],len(results_layers))
        else:
            print(layer['class_name'],len(results_layers))
            print("error")
    return results_layers[-1]
def relu(image):
    return image.gt(0).multiply(image)

In [ ]:
l8 = ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')
shp = r'[filename]'
#####################################
# filename of a boundary shapefile
#####################################
baikal = geemap.shp_to_ee(shp)
task = {}
for st_year in range(2013,2021):
    ed_year = st_year
    DEM_0 = ee.Image('CGIAR/SRTM90_V4').reproject(crs = 'EPSG:4326',scale=30)
    DEM = DEM_0.divide(4000)
    imagec = summer_img_bounds(l8, st_year, ed_year, baikal)
    image_freecloud = imagec.map(cloud_free).mean().reproject(crs = 'EPSG:4326',scale=30)
    image = image_freecloud.select('SR_B.').addBands(DEM.clip(baikal)).reproject(crs = 'EPSG:4326',scale=30)
    image = image.setDefaultProjection(image.projection())
    image = features(image)
    result_img = pixel_based_CNN(image)
    task[str(st_year)] = ee.batch.Export.image.toAsset(**{
      'image': result_img.select('B').gt(result_img.select('B_1')),
      'description': 'baikal_water'+str(st_year),
      'assetId': '[folder]'+str(st_year),
        ####################################
        # folder in assets
        ####################################
      'scale': 30,
      'region': baikal.geometry(),
      'maxPixels':10000000000
    })
    task[str(st_year)].start()